In [1]:
import json
import urllib
import urllib.parse
import os
import pandas as pd

In [6]:
scopus_conferences = ["CEP", "UKICER", "Koli", "ITiCSE", "ICER"]

data_dir = "data"

In [18]:
def affil_detail(affil):
#    print ("affil_detail for ", affil)
    return affil['affilname']

In [42]:
affiliations = []

paper_dfs = []

for conference in scopus_conferences:
    with open(conference + "_papers.json") as conff:
        conf_paper_files=json.load(conff)
        for paper_fname in list(conf_paper_files):
            fname = urllib.parse.quote(paper_fname, safe='')
            print ("opening ", fname)
            with open(os.path.join(data_dir, fname + ".json")) as paperf:
                paper = json.load(paperf)
                paper_dict = {}
                try:
                    paper_dict['conference'] = conference
                    date = paper['coredata']['prism:coverDate']
                    year = date[0:4]
                    paper_dict['year'] = [year]
                    paper_dict['title'] = [paper['coredata']['dc:title'].strip()]
                    paper_dict['doi'] = [paper['coredata']['prism:doi']]
                    paper_dict['scopus_url'] = [paper['coredata']['prism:url']]

                    title_hash = str(hash(str(paper_dict['title'][0])))
                    paper_dict['title_hash'] = [title_hash]
                    paper_dict['key'] = "_".join([conference,year , title_hash[-6:]])
                    paper_affil = paper['affiliation']
                    if isinstance(paper_affil, list):
                        affil_details =  list(map(affil_detail, paper_affil))
                        affiliations = affiliations + affil_details
                    else:
                        affil_d = affil_detail(paper_affil)
                        affiliations.append(affil_d)
#                    paper_dict['abstract'] = [paper['coredata']['dc:description']]

                except KeyError as e:
                    print("KeyError ", e, " in ",fname)
            
                paper_df = pd.DataFrame(paper_dict)
                paper_dfs.append(paper_df)


opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219210026
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219206346
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219203202
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219203129
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219195595
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219193620
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219189082
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219187395
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219171503
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219170874
opening  https%3A%2F%2Fapi.elsevier.com%2Fcontent%2Fabstract%2Fscopus_id%2F85219168230
opening  https%3A%2F%2Fapi.elsevier.com%2Fc

FileNotFoundError: [Errno 2] No such file or directory: 'ICER_papers.json'

In [44]:
papers_df = pd.concat(paper_dfs)
papers_df.set_index('key', inplace=True)
papers_df

,conference,year,title,doi,scopus_url,title_hash
key,,,,,,
CEP_2025_671693,CEP,2025,Enhancing conceptual understanding in early ye...,10.1145/3702212.3702220,https://api.elsevier.com/content/abstract/scop...,-7612675003404671693
CEP_2025_425228,CEP,2025,Assessing Software Engineering Students' Analy...,10.1145/3702212.3702223,https://api.elsevier.com/content/abstract/scop...,6668210468012425228
CEP_2025_456639,CEP,2025,Themes in the Declared Use of Generative Artif...,10.1145/3702212.3702217,https://api.elsevier.com/content/abstract/scop...,-6489872609726456639
CEP_2025_892833,CEP,2025,Learning Programming Languages by Pantomime,10.1145/3702212.3702213,https://api.elsevier.com/content/abstract/scop...,5183681864324892833
CEP_2025_871360,CEP,2025,FLARE: A Framework Supporting Code Comprehensi...,10.1145/3702212.3702219,https://api.elsevier.com/content/abstract/scop...,-6801671759654871360
...,...,...,...,...,...,...
ITiCSE_2019_060300,ITiCSE,2019,A conceptual framework for identifying emotion...,10.1145/3304221.3325570,https://api.elsevier.com/content/abstract/scop...,8589750483465060300
ITiCSE_2019_448829,ITiCSE,2019,A reflective writing framework for computing e...,10.1145/3304221.3325580,https://api.elsevier.com/content/abstract/scop...,314100104685448829
ITiCSE_2019_683039,ITiCSE,2019,Devising Work-based Learning Curricula with Ap...,10.1145/3304221.3325576,https://api.elsevier.com/content/abstract/scop...,-5614900661678683039


In [45]:
affiliations

['University of Glasgow',
 'The Open University',
 'Swansea University',
 'University of Strathclyde',
 'University of Glasgow',
 'University of Strathclyde',
 'Arunside Primary School',
 'Swansea University',
 'University of Glasgow',
 'University of West London',
 'Abertay University',
 'Swansea University',
 'Swansea University',
 'Altitude Foundation',
 'The University of Manchester',
 'The University of Edinburgh',
 'Durham University',
 'Manchester Metropolitan University',
 "Queen's University Belfast",
 'Colgate University',
 'University of Nebraska–Lincoln',
 'University of Cambridge',
 'Raspberry Pi Foundation',
 'The Open University',
 'Swansea University',
 'University of Glasgow',
 'University of Strathclyde',
 'Swansea University',
 "King's College London",
 'University of Strathclyde',
 'University of Glasgow',
 'University of Glasgow',
 'University of Strathclyde',
 "Queen's University Belfast",
 'University of Strathclyde',
 'University of Glasgow',
 'The Open Universi

In [121]:
with open("2018_CEP_papers.json") as CEP_2018f:
    CEP_2018_papers=json.load(CEP_2018f)
    
CEP_2018_papers[3].keys()

dict_keys(['time', 'authors', 'title', 'abstract'])

In [122]:
for CEP_2018_paper in CEP_2018_papers:
    paper_dict = {}
    paper_dict['title'] = [CEP_2018_paper['title'].strip()]
    print ("adding ", paper_dict['title'])
    paper_dict['abstract'] = [CEP_2018_paper['abstract']]
    paper_dict['date'] = ['2018-01-11']
    paper_df = pd.DataFrame(paper_dict)
    paper_dfs.append(paper_df)

    
CEP_2018_papers

adding  ['Intelligent Agents for Education']
adding  ['Sustainable Computing: a home for LSEP, Commercial practice and Research skills']
adding  ['Including IT service management in the Computing curriculum: a caricature approach.']
adding  ['"It\'s only a computing project - so there can\'t be any ethical issues..."']
adding  ['No money worries: new student identities for Graduate Level Apprentices']
adding  ['The enemies within - inhibitors to learning']
adding  ['Blocks-based programming for introductory programming in Python']
adding  ['Enhancing computer programming skills through Virtual Reality']
adding  ['Supporting the learning of programming with the PRIMM approach']
adding  ['Self regulated Learning in Engineering and Computing Classrooms']
adding  ['Learner engagement in larger computer science classes']
adding  ["Promoting learning by going against students' preference to learning"]
adding  ['The Associate Student Project celebrates five years of student transitions from f

[{'time': '',
  'authors': [{'firstName': 'Karen',
    'lastName': 'Hunsdale',
    'email': '',
    'department': '',
    'institution': 'University of Winchester\t'}],
  'title': 'Intelligent Agents for Education',
  'abstract': "The use of CAs can be applied to educational settings to better support teaching and learning and alternative learning environments. Computing underpins almost all areas of the modern world and many new opportunities in science and engineering could not have been realized without it. Thus, computer scientists have the potential to revolutionize societies, develop economic wealth and change people’s lives. Employers needs to ensure that its future generations have the skills that employers require, which is only achieved through effective teaching and learning practices at our schools. If we do not engage at this level, we are unlikely to produce the number of computer scientists required to satisfy the demands of industry. Teachers need support from the scien

In [129]:
papers_df = pd.concat(paper_dfs, ignore_index=True)

In [62]:
with open("abstracts.json", "w") as abs_f:
    json.dump(abstracts, abs_f, indent=4)

In [125]:
len(papers_df)

103

In [130]:
papers_df.to_csv("CEP_papers.csv")